In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as pl

# Execute all the cells below...

In [4]:
import base64
from pyquery import PyQuery
import unicodedata

In [5]:
def unicode_normalize(text):
    return unicodedata.normalize('NFKD', unicode(text)).encode('ascii','ignore')

In [6]:
def parse_email(email_body):
    email_parser = PyQuery(email_body)
    email_fields = [unicode_normalize(span.text) for span in email_parser('div>span')]
    
    feature_names = ['remetente', 'email', 'assunto', 'tipo', 'data_envio', 'respondida?', 'mensagem_externa?']
    features = dict(zip(feature_names, email_fields))
    
    features['mensagem'] = unicode_normalize(email_parser('div>div').text()).lower()
    return features

In [7]:
messages = []
input_file = open('INBOX.Arquivo Remoto.mbox')
line = input_file.readline()
while line:
    if 'X-Proofpoint-Virus-Version' in line: 
        content = ''
        input_file.readline()
        line = input_file.readline()
        line = ''
        while line != '\n':
            line = input_file.readline()
            content += line
        messages.append(content)
    line = input_file.readline()   

decoded_emails = []
for m in messages:
    try:
        decoded_message = [base64.b64decode(x) for x in m.split('\n')]
        decoded_message = ''.join(decoded_message)
        decoded_emails.append(parse_email(decoded_message))
    except:
        pass

# ... and from now on, *decoded_messages* contains the emails as a list of dicts :D